# Modelling and Evaluation

**Make sure that you have run data_collection.ipynb before running this one.**

## Objectives

- To address business requirement 2:
  - The client wants to know if a given image of a cherry leaf is healthy or shows powdery mildew.

## Inputs

This notebook will use the following inputs:

- inputs/cherry-leaves/cherry-leaves/test
- inputs/cherry-leaves/cherry-leaves/train
- inputs/cherry-leaves/cherry-leaves/validation
- outputs/v1/image_shape.pkl

## Outputs

This notebook will generate the following outputs:

- Plot showing the number of images in the train, validation, and test sets.
- Image augmentation.
- Class indices to change prediction inference in labels.
- Machine learning model creation and training.
- Saved model.
- Learning curve plot for model performance.
- Model evaluation on pickle file.
- Prediction on the random image file.

## Import packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import joblib
import pickle
import tensorflow
from matplotlib.image import imread
from PIL import Image
from pathlib import Path


## Change working directory

By default, the working directory is "jupyter_notebooks", where the notebook is running. However, we need to change the working directory to its parent folder so that file references align with the broader project structure.

To do this, we first check the current working directory — note that the output below only displays the last two folders in the file path, rather than the full system path. This is done intentionally to prevent exposing the full local file path stored on my machine.

**Any time you revisit this notebook after logging out, or open a different notebook for the first time, you must repeat these steps to ensure the working directory is always correctly set.**

In [ ]:
# Get the current directory
current_dir = os.getcwd()
current_dir


Now we change the working directory from "jupyter_notebooks" to the parent directory.

In [ ]:
# Change the working directory to its parent folder
os.chdir(os.path.dirname(os.getcwd()))

# Confirmation message with
print("You set a new current directory")


Confirm the new current directory.

In [ ]:
# Get the current directory
current_dir = os.getcwd()
current_dir


## Set the input directories

Now we need to define the paths where this notebook will find the data we'll be using.

In [ ]:
# Define base image data path
image_data = Path("inputs") / "cherry-leaves" / "cherry-leaves"

# Define paths for train, validation, and test sets
train_path = image_data / "train"
val_path = image_data / "validation"
test_path = image_data / "test"

# Print paths
print(f"Train Path: {train_path}")
print(f"Validation Path: {val_path}")
print(f"Test Path: {test_path}")


### Confirm that the file paths are valid

Now we'll run a function just to check that the file paths defined above are valid.

The function will select one image from each train, test and validation folder and display it below along with the file path beginning with the "inputs" folder.


In [ ]:

def show_random_image(parent_folder):
    """
    Selects and displays a random image from a subfolder inside the given
    parent directory.

    Args:
        parent_folder (str | Path): Path to dataset folder (e.g., train, val).

    Returns:
        str | None: Path to the randomly selected image or None if not found.
    """

    parent_folder = Path(parent_folder)  # Ensure it's a Path object

    # Ensure the parent folder exists
    if not parent_folder.exists():
        print(f"path does not exist: {parent_folder}")
        return None

    # Scan subdirectories ('healthy', 'powdery_mildew')
    subfolders = [f for f in parent_folder.iterdir() if f.is_dir()]

    if not subfolders:
        print(f"no subfolders found in: {parent_folder}")
        return None

    # Pick a random subfolder (class name)
    chosen_folder = random.choice(subfolders)

    # Find image files inside that subfolder
    img_exts = {'.png', '.jpg', '.jpeg', '.bmp', '.webp'}
    image_files = [
        f for f in chosen_folder.iterdir()
        if f.suffix.lower() in img_exts
    ]

    if not image_files:
        print(f"no images found in: {chosen_folder}")
        return None

    # Select a random image and display it
    chosen_image = random.choice(image_files)
    image = Image.open(chosen_image)

    # Extract folder name ('healthy'/'powdery_mildew') and dataset section
    class_name = chosen_folder.name
    dataset_section = parent_folder.name

    # Display the image with correct labeling
    plt.imshow(image)
    plt.title(f"'{class_name}' image from '{dataset_section}' folder")
    plt.axis("off")  # Hide axes for a cleaner look
    plt.show()

    return str(chosen_image)  # Return path for confirmation


# Fetch and display random images from each dataset section
train_image = show_random_image(train_path)
print(f"random train image: {train_image}")

val_image = show_random_image(val_path)
print(f"random validation image: {val_image}")

test_image = show_random_image(test_path)
print(f"random test image: {test_image}")


### Set Output Directory

Create 'output' directory if it doesn't already exist.

In [ ]:

# Define the root directory
root_dir = Path.cwd()  # Get current working directory
version = "v1"

# Construct the relative path for outputs
file_path = Path("outputs") / version

# Check if the directory for this version already exists
if file_path.exists():
    print(f'Old version "{version}" already exists.')
    print(f'Create a new version if required. If not, you are good to go!')
else:
    try:
        # Create the directory if it doesn't exist
        file_path.mkdir(parents=True, exist_ok=True)
        print(f'Created new directory: {file_path}')
    except OSError as e:
        print(f'Error creating directory: {e}')


## Set the Label Names

Our upcoming binary classification task will require labelled data in order to train. Here we will set the labels to be used in our project:

- 'healthy'
- 'powdery_mildew'


In [ ]:
# Set the labels
labels = os.listdir(train_path)
print('The labels for the images are', labels)


## Let's Count the Images

In the Data Collection notebook, we downloaded two directories of images:

- healthy
- powdery_mildew
  
We then split the images in each directory into three further folders:

- test
- train
- validation

In this notebook, we now need to count how many images are in each of these six folders. We will create a loop to do the count then display the results in a plot.

In [ ]:

# Create an empty dictionary
data = {'Set': [], 'Label': [], 'Frequency': []}

# Define dataset partitions
folders = ['train', 'validation', 'test']

# Define root dataset directory
image_data = Path("inputs/cherry-leaves/cherry-leaves")
# Extract class labels dynamically from 'train' subdirectories
train_path = image_data / "train"

# Extract directory names
labels = [
    label.name for label in train_path.iterdir()
    if label.is_dir()
]

# Collect data efficiently
for folder in folders:
    for label in labels:
        label_path = image_data / folder / label  # Use Path object

        # Count files if path exists
        num_images = (
            len(list(label_path.glob("*"))) if label_path.exists()
            else 0
        )

        data['Set'].append(folder)
        data['Label'].append(label)
        data['Frequency'].append(num_images)

        print(f"{folder} - {label}: {num_images} images")

# Convert to DataFrame
df_freq = pd.DataFrame(data)

# Create the figure object
fig, ax = plt.subplots(figsize=(8, 5))

# Define custom colors
custom_colors = [
    "#582088", "#33FFC29F"
]

# Plot with updated colors
sns.set_style("whitegrid")
sns.barplot(
    data=df_freq, x='Set', y='Frequency', hue='Label',
    palette=custom_colors, ax=ax
)

# Show plot (figure remains stored in 'fig' for later saving)
plt.show()


### Save the plot

If you're happy with the plot layout and colour scheme, save it in the 'outputs/v1' folder.

In [ ]:
save_path = Path(file_path) / "labels_distribution.png"
fig.savefig(save_path, bbox_inches="tight", dpi=150)

print("Plot saved successfully!")


As we can see from the count, we have equal numbers of 'healthy' and 'powdery_mildew' images across the train, test and validation folders. Had the numbers been unequal, we would have given consideration to data balancing (oversampling or undersampling). As it stands, we don't need to do any data balancing so we can now move on to data augmentation.


## Data Augmentation

Currently all our images look roughly the same in terms of size, aspect and zoom, for example. To build a more robust machine learning model we need to 'rough up' our dataset a little. Real world images won't always look like the uniform set of images that we downloaded so we need the model to learn using an 'augmented' dataset. An augmented dataset introduces variations such as random rotations, cropping, zoom adjustments, brightness shifts, and flipping to simulate the unpredictability of real-world images.

This process helps the model to generalize better by learning from a diverse set of image transformations. By applying augmentation techniques, we can effectively enhance the robustness of our machine learning model, making it more resilient to variations in new and unseen data.

So, let's get to it!


### Import ImageDataGenerator

First we'll import the right packages from tensorflow.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image generator with normalization values for pixels
image_gen = ImageDataGenerator(rescale=1./255)

print("Image generator initialized.")


Let's now define the adjustments that we want to make. Credit Code Institute WalkthroughProject01 for initial parameter selection.

In [ ]:
# Define image data generator with various augmentations
augmented_image_data = ImageDataGenerator(

    # Rotation to introduce perspective shifts
    rotation_range=25,

    # Shift image horizontally by 15% of image width
    width_shift_range=0.15,

    # Shift image vertically by 15% of image height
    height_shift_range=0.15,

    # Slants the image along both axes by up to 20%
    shear_range=0.2,

    # Zoom in or out by up to 20%
    zoom_range=0.2,

    # Flip images horizontally
    horizontal_flip=True,

    # Flip images vertically
    vertical_flip=True,

    # Brightness adjusts between 70% - 130%
    brightness_range=(0.7, 1.3),

    # Shift RGB channels by 20 units for variance
    channel_shift_range=20,

    # Fill missing pixels with nearest neighbor interpolation
    fill_mode='nearest',

    # Normalize pixel values to scale between 0 and 1
    rescale=1./255
)


### Augmentation notes

We are going to be augmenting the image data in real-time by using ImageDataGenerator's flow_from_directory() method. This means that the images will be modified dynamically as they enter the model. This ensures that the model sees new variations of images in every epoch, helping it learn generalized patterns rather than memorizing specific features.

In the data visualization notebook, we calculated the average image shape and stored that value in 'image_shape.pkl'. The model that we will be developing will need to know the dimensions of the images we will be training. To reduce the size of the .keras file that we will be generating, we will be defining a custom image shape in the cell block.

We may experiment with batch size as we fine-tune the model. Adjusting batch size impacts training speed and memory efficiency, so we may modify this value, if needed, to optimize performance.


### Define the image shape manually

In [ ]:
# Define image shape manually
image_shape = (128, 128, 3)  # Example: (height, width, channels)

print(f"Using manually defined image shape: {image_shape}")


### Augment the train image data

In [ ]:

# Define batch size for training
# 8 balances memory efficiency & training speed after trial and error
batch_size = 8

# Convert train_path to a Path object
train_path = Path("inputs/cherry-leaves/cherry-leaves/train")
if not train_path.exists():
    raise FileNotFoundError(f"Training directory {train_path} not found!")

# Extract width & height from stored image shape
target_size = tuple(image_shape[:2])

# Load training dataset with augmentation
train_set = augmented_image_data.flow_from_directory(
    train_path,
    target_size=target_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True
)

# Check dataset summary
print(f"Using batch size: {batch_size}")
print(f"Found {train_set.samples} images belonging to {
    len(train_set.class_indices)} classes.")
print(f"Class mapping: {train_set.class_indices}")

# Load a sample batch to verify the image size
sample_images, _ = next(train_set)  # Fetch first batch

# Expected: (batch_size, {target_size[0]}, {target_size[1]}, 3)
print(f"Sample image batch shape: {sample_images.shape}")


### Augment test image data

Now, when it comes to augmenting test image data, we are only going to worry about resizing the images. This is because the test images should be as close as possible to the 'real world' images that will be uploaded by the users.

As you can see from the variable below, we are only going to make use of the rescale parameter in the ImageDataGenerator class.

In [ ]:

# Extract width & height
target_size = tuple(image_shape[:2])

# Define the correct test path
test_path = Path("inputs/cherry-leaves/cherry-leaves/test")

# Validate path existence to prevent errors
if not test_path.exists():
    raise FileNotFoundError(f"Test directory {test_path} not found!")

# Load test dataset with augmentation
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    str(test_path),  # Ensure compatibility by converting Path to string
    target_size=target_size,
    color_mode="rgb",  # Loads images in RGB format
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False  # Keeps data order unchanged for consistent evaluation
)

# Check dataset summary
print(f"Using batch size: {batch_size}")  # Confirm batch size
print(f"Found {test_set.samples} images belonging to {
    len(test_set.class_indices)} classes.")
print(f"Class mapping: {test_set.class_indices}")

# Load a sample batch to verify image size
sample_images, _ = next(test_set)  # Fetch first batch

# Expected: (batch_size, {target_size[0]}, {target_size[1]}, 3)
print(f"Sample image batch shape: {sample_images.shape}")


### Augment validation image data

The same logic applies to the validation image data. This should also match 'real world' data as closely as possible.

Notice that the basic code for augmenting the validation data is the same as that for the test data.

In [ ]:
from pathlib import Path

# Extract width & height
target_size = tuple(image_shape[:2])

# Define the correct validation path
val_path = Path("inputs/cherry-leaves/cherry-leaves/validation")

# Validate path existence to prevent errors
if not val_path.exists():
    raise FileNotFoundError(f"Validation directory {val_path} not found!")

# Load validation dataset
val_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    str(val_path),  # Convert Path to string
    target_size=target_size,
    color_mode="rgb",  # Loads images in RGB format
    batch_size=batch_size,  # Define batch size
    class_mode="binary",
    shuffle=False  # Ensures consistent evaluation during validation
)

# Print dataset summary
print(f"Using batch size: {batch_size}")
print(f"Found {val_set.samples} images belonging to {
    len(val_set.class_indices)} classes.")
print(f"Class mapping: {val_set.class_indices}")

# Load a sample batch to verify image size
sample_images, _ = next(val_set)  # Fetch first batch

# Expected: (batch_size, {target_size[0]}, {target_size[1]}, 3)
print(f"Sample image batch shape: {sample_images.shape}")


### Save class indices for later use

Let's now save the class indices into a .pkl file for later use. This will be required when deploying our model to ensure class labels remain consistent during inference (making predictions on unseen data).

In [ ]:

# Ensure file_path is a Path object
file_path = Path(file_path)

# Save class indices to a file using Pathlib
output_file = file_path / "class_indices.pkl"
joblib.dump(value=train_set.class_indices, filename=output_file)

print(f"File saved: '{output_file}'")


### Let's take a look at some examples the augmented images

Before we start modelling, let’s take a look at some samples of the augmented image data. The function below can be used to pull some random images from the directories.


In [ ]:
def show_random_augmented_samples(train_set, val_set, test_set):
    """Displays three randomly selected augmented images from each dataset 
    (train, validation, test) in a 3x3 plot.

    Args:
        train_set (tf.data.Dataset): Augmented training dataset.
        val_set (tf.data.Dataset): Augmented validation dataset.
        test_set (tf.data.Dataset): Augmented test dataset.
    """

    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    dataset_names = ["Train", "Validation", "Test"]
    datasets = [train_set, val_set, test_set]

    # Iterate through datasets with names for plotting
    for col, (dataset_name, dataset) in enumerate(
        zip(dataset_names, datasets)
    ):
        # Generate random indices for batch selection
        random_indices = np.random.randint(0, dataset.batch_size, 3)

        for row, idx in enumerate(random_indices):
            # Select images at random positions in batch
            img, label = next(dataset)  # Fetch a batch
            img = img[idx]  # Extract random image from batch

            # Display the images in the subplot
            axes[row, col].imshow(img)  
            # Hide axis for cleaner view
            axes[row, col].axis("off")  

            if row == 0:
                axes[row, col].set_title(
                    dataset_name, fontsize=14, fontweight="bold"
                )

    plt.tight_layout()
    plt.show()

Now call the function...

In [ ]:
show_random_augmented_samples(train_set, val_set, test_set)

## The Model

Now we need to create a model that trains on our augmented dataset. This training will seek to identify patterns in the images and accurately classify cherry leaves as either healthy or affected by powdery mildew. 

This next step involves designing a deep learning model.


### Import the right tensor packages



In [ ]:
# Import TensorFlow components for model training
from tensorflow.keras.preprocessing.image import ImageDataGenerator, \
    img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Activation, Dropout, Flatten, Dense,
                                     Conv2D, MaxPooling2D, Input,
                                     BatchNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers

print("TensorFlow methods imported successfully.")


### Define the model

The following [model](https://github.com/Duffew/mildew_detector#the-model) is a sequential convolutional neural network designed to perform a binary classification task; learning to differentiate between healthy and mildewed cherry leaves.  

Credit Code Institute WalkthoughProject01 for the initial model structure.  


In [ ]:
def create_tf_model(input_shape=image_shape):
    """
    Create a sequential Convolutional Neural Network for binary
    classification of leaf images.

    Args:
        input_shape (tuple): Image shape (height, width, channels).

    Returns:
        Sequential: Compiled TensorFlow model ready for training.
    """
    model = Sequential()

    model.add(layers.Conv2D(
        32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())

    # L2 added to help prevent overfitting by penalizing large weight values
    # Added in response to validation accuracy issues in
    # previous model iterations
    model.add(layers.Dense(128, activation='relu',
                           kernel_regularizer=l2(0.0005)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


#### Create a model summary

We do this so that we can visually inspect the model architecture, verify layer configurations, check input/output shapes, and ensure the number of trainable parameters aligns with our expectations before training begins.


In [ ]:
create_tf_model().summary()


#### Early stopping

As the model learns it should get more accurate over the epochs. At some point, the learning will slow and improvements will become minimal. Once this begins to happen, we can stop the training early to help prevent overfitting, ensuring the model does not memorize the training data but generalizes well on unseen data.

The variable below can be used to stop the training when after 3 epochs (patience=3), value loss stops improving (monitor='val_loss'). This variable will also restore the weights from the best performing epoch (restore_best_weights=True) which will then become the inference model used by the client.


In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True)


Before fitting the model, let’s just check how the training data is broken down.


In [ ]:
print(f"Total training images: {train_set.samples}")
print(f"Batch size: {batch_size}")
print(f"Calculated steps_per_epoch: {train_set.samples // batch_size}")


### Fit the model

Run the cell below to fit the model.

In [ ]:
# Fit model for training

# Create the model
model = create_tf_model()

# Train the model
steps_per_epoch = train_set.samples // batch_size

model.fit(
    train_set,
    epochs=25,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_set,
    callbacks=[early_stop],
    verbose=1
)


#### Save model

Now we save the model with the best weights to a .keras file.

In [ ]:
save_path = "outputs/v1/mildew_detector_model.keras"
model.save(save_path)

# Get file size in MB
file_size_mb = os.path.getsize(save_path) / (1024 * 1024)

print(f"File saved: '{save_path}' | Size: {file_size_mb:.2f} MB")


#### Model performance

Now that the model has been fitted, we need to assess its performance. The code below will generate a couple of plots to show the training loss compared to the validation loss, and the training accuracy compared to the validation accuracy.


In [ ]:
# Define custom colours to apply to the plots
custom_colors = ["#582088", "#33FFC29F"]

# Define source of performance data
perform_data = pd.DataFrame(model.history.history)

sns.set_style("whitegrid")

# Loss plot using custom colours and saved in v1
perform_data[['loss', 'val_loss']].plot(style='.-', color=custom_colors)
plt.title("Loss")
plt.savefig(f'{
    file_path}/model_training_losses.png', bbox_inches='tight', dpi=150)
plt.show()

print("\n")

# Accuracy plot using custom colours and saved in v1
perform_data[['accuracy', 'val_accuracy']].plot(
    style='.-', color=custom_colors)
plt.title("Accuracy")
plt.savefig(f'{
    file_path}/model_training_acc.png', bbox_inches='tight', dpi=150)
plt.show()


The following code will display the saved plots so we can see the performance - use this code block if you don't want to re-run the model but have access to the saved plots in outputs/v1.


In [ ]:

# Define the output folder
output_dir = Path("outputs/v1")  # Adjust if needed

# List of saved plots
plot_files = ["model_training_acc.png", "model_training_losses.png"]

# Display each plot
for plot_file in plot_files:
    plot_path = output_dir / plot_file  # Construct the full path
    if plot_path.exists():
        img = plt.imread(plot_path)
        plt.imshow(img)
        plt.axis("off")
        plt.title(plot_file)
        plt.show()
    else:
        print(f"File not found: {plot_path}")


#### Performance summary

The above plots show that validation accuracy started high and remained high across all epochs. It stayed consistently above training accuracy, which started low but jumped quickly above 94% and peaked at 98.86% in epoch 6. This suggests rapid learning early in the training. While validation accuracy remained stable, there were slight fluctuations in later epochs that might warrant further investigation.

Similarly, the loss values for the training data show rapid declines as the epochs progressed. Validation loss peaked in the second epoch at 0.1441, then dropped and mostly stayed below 0.1, with slight fluctuations. This suggests confident generalization. As validation losses plateaued, it is likely that training stopped early due to patience becoming exhausted - early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True).

These results suggest that this model is high performing, learns quickly and maintains excellent validation accuracy. Although there are some slight variations in validation loss, the model seems to generalise well 


### Model evaluation

Run the following code cell to evaluate the fitted model using the test set image data.

In [ ]:
evaluation = model.evaluate(test_set)


Now save the evaluation in a .pkl file for later use.

In [ ]:
joblib.dump(value=evaluation,
            filename=f"outputs/v1/evaluation.pkl")


#### Predict on new data

The code block below will select and load an image from the test set to be used by the model to infer classification.

If you want to change the image, adjust the 'pointer' value. If you want test on a different category, adjust the 'label' value.

In [ ]:
from tensorflow.keras.preprocessing import image

# Define pointer & label - adjust pointer number to see different test images
pointer = 42
label = labels[1]  # Select "healthy" - [0] or "mildewed" - [1]

# Get image directory and file list
image_dir = Path(test_path) / label
image_files = list(image_dir.iterdir())

# Ensure pointer is within range
if pointer >= len(image_files):
    raise IndexError(f"Pointer index {
        pointer} exceeds available images({len(image_files)}).")

# Load image
image_path = image_files[pointer]
pil_image = image.load_img(
    image_path, target_size=image_shape, color_mode='rgb')

# Display image details
print(f"Image Path: {image_path}")
print(f"Image Shape: {pil_image.size}, Image Mode: {pil_image.mode}")

# Show image
plt.imshow(pil_image)
plt.axis("off")
plt.title(image_path.name)
plt.show()


Now prepare the image for inference.

In [ ]:
# Convert the PIL image to a NumPy array
my_image = image.img_to_array(pil_image)  # Converts RGB image into an array

# Expand dimensions to match the expected input shape for the model
my_image = np.expand_dims(my_image, axis=0)  # Adds a batch dimension

# Normalize pixel values to range [0,1] for better model performance
my_image = my_image / 255.0  # Standard normalization for neural network input

# Print the shape to verify transformation
print(f"Processed Image Shape: {my_image.shape}")
print(f"Training Image Shape: {image_shape}")
print("Processed image shape should match training image shape.")
print("Don't worry about the first number. It's the batch size for testing.")


#### Inference

Run the following cell to predict the classification of the test image and see the model's confidence in that prediction. This code takes the numeric probability and turns it into a binary classification.

In [ ]:
# Predict raw probability
raw_output = model.predict(my_image)[0, 0]

# Map numerical predcitions to 'healthy' or 'powdery_mildew'
target_map = {v: k for k, v in train_set.class_indices.items()}

# Determine predicted class - apply explicit type conversion at a 50% threshold
pred_class = target_map[int(raw_output > 0.5)]

# Adjust probability for binary classification
pred_proba = np.abs(raw_output - (pred_class == target_map[0]))

# Print results
print(f"Predicted Class: {pred_class}")  # Classification
print(f"Raw Model Output: {raw_output:.9f}")  # Raw output to 9 deciaml places
print(f"Confidence: {pred_proba:.4f}")  # Confidence to 4 deciaml places


#### Evaluation summary

The model was evaluated using a set of unseen test data, split into 106 batches.

The evaluation returned an accuracy of 0.9965 and a loss of 0.0909. Both metrics are in line with the validation figures observed during model fitting.

Additionally, the model correctly identified an image of powdery mildew with a confidence level of 0.9998.

Taken together, these results provide strong evidence that the model is highly accurate and reliable in detecting powdery mildew in cherry leaves. The model exceeds the client's requirement of 97% accuracy and is well-suited for deployment.


## Summary

That was a lot of work! Well done on getting this far! In this notebook we:

- Counted the images in each folder and created a plot to show the numbers
- Augmented the train, validation and test sets for training
- Created a ML model to train on the data
- Saved an ML model with the best weights
- Assessed model performance in terms of 'accuracy' and 'loss'
- Evaluated the saved model for 'accuracy' on an unseen test set
- Predicted leaf health on unseen images

Definitely time for a cup of tea before we put all this into a dashboard for our client!
